In [16]:
%load_ext autoreload
%autoreload 2


import numpy as np
from utils.data_helper import *
from utils.data import *
from utils.stats import *
from utils.performance import *
from plotly.subplots import make_subplots
from account import Binance
import pandas as pd
import warnings
from strategy_v3.Strategy import *
from strategy_v3.Executor import ExecutorBinance, ExecutorBacktest
from strategy_v3.ExecuteSetup import *
from strategy_v3.DataLoader import DataLoaderBinance
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from time import sleep


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 30)
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
symbol = 'BTCFDUSD'
binance = Binance()
df = binance.get_historical_instrument_price(symbol, interval='1m', start_str='2 hours ago')
plot_price_ohcl(df, symbol)
df['std'] = df['Close'].rolling(10).std()

# Market Making

In [ ]:
symbol = 'BTCUSDT'
df_bids, df_asks = Binance().get_order_book(instrument=symbol, limit=200)
spread = df_asks.iloc[0]['price'] - df_bids.iloc[0]['price']

fig = make_subplots()
fig.update_layout(title=f'{symbol} (spread = {spread:.4f})', width=800, height=500, hovermode='x')
fig.add_trace(go.Scatter(x=df_bids['price'], y=df_bids['quantity_cum'], name='bid'))
fig.add_trace(go.Scatter(x=df_asks['price'], y=df_asks['quantity_cum'], name='ask'))


current_position = 0.0007*5
target_position = 0
vol = df['std'].iloc[-1] 
gamma = 0.4

mid_price = (df_asks.iloc[0]['price'] + df_bids.iloc[0]['price']) / 2
r = mid_price - (current_position - target_position) * gamma * vol ** 2
spread = 20
bid = r - spread/2
ask = r + spread/2

fig.add_vline(bid, line_dash="dash", line_color="blue")
fig.add_vline(ask, line_dash="dash", line_color="red")

fig.show()
print(spread)

20


In [160]:
strategy = SimpleMarketMakingStrategy(
    instrument = 'BTCFDUSD',
    interval = '1m',
    vol_lookback = 20,    
    gamma = 0.4,                
    spread = 20,       
    target_position = 0,                
    position_size = 10,
    price_decimal = 2,
    qty_decimal = 5,                             
    status = STATUS.RUN,
    verbose = True,       
    start_date = '2024-03-06 12:00:00'
)
strategy.set_data_loder(DataLoaderBinance())
strategy.set_executor(ExecutorBinance())
strategy.set_strategy_id('SMM_BTCv1')    

In [136]:
try:
    while True:  
        strategy.load_data('2 Hours Ago')                                
        df = strategy.df
        data = df.iloc[-1]                
        strategy.execute(data)    
        sleep(30)
except Exception as e:    
    strategy.logger.error(e)
    strategy.cancel_all_orders()
    strategy.close_out_positions()

2024-03-08 03:22:18,070 - smm_SMM_BTCv1 - INFO - inv: 0.0, mid: 67240.05, r: 67240.05, spread: 20. creating new bid ask orders.....
2024-03-08 03:22:48,764 - smm_SMM_BTCv1 - INFO - inv: 0.0, mid: 67248.34, r: 67248.34, spread: 20. creating new bid ask orders.....
2024-03-08 03:23:19,455 - smm_SMM_BTCv1 - INFO - inv: 0.0, mid: 67218.34, r: 67218.34, spread: 20. creating new bid ask orders.....
2024-03-08 03:23:50,208 - smm_SMM_BTCv1 - INFO - inv: 0.00015, mid: 67163.74, r: 67162.8, spread: 20. creating new bid ask orders.....
2024-03-08 03:24:20,976 - smm_SMM_BTCv1 - INFO - inv: 0.0003, mid: 67107.37, r: 67104.79, spread: 20. creating new bid ask orders.....
2024-03-08 03:24:52,076 - smm_SMM_BTCv1 - INFO - inv: 0.0003, mid: 67201.82, r: 67199.25, spread: 20. creating new bid ask orders.....
2024-03-08 03:25:22,740 - smm_SMM_BTCv1 - INFO - inv: 0.0003, mid: 67197.56, r: 67194.69, spread: 20. creating new bid ask orders.....
2024-03-08 03:25:53,399 - smm_SMM_BTCv1 - INFO - inv: 0.0003, mi

KeyboardInterrupt: 

In [162]:
strategy.cancel_all_orders()
strategy.close_out_positions()

2024-03-09 01:41:52,305 - smm_SMM_BTCv1 - INFO - cancelling all orders.....
2024-03-09 01:41:52,956 - smm_SMM_BTCv1 - INFO - nothing to close out because of no oustanding positions....


In [174]:
strategy.load_data('24 Hours Ago', lookback_end='12 Hours Ago')        

In [175]:
strategy.summary()

,Measure,smm_SMM_BTCv1
0,Pnl,0.010683
1,Trading Fee,0.000000
2,Cumulative Return,1.000534
3,Annualized Return,0.384588
4,Annualized Volatility,0.344634
5,Annualized Sharpe Ratio,0.970083
6,Maximum Drawdown,-0.018519
